# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Del = (comms[1]*comms[2]*comms[3]*Word.new('s')).freeze
p 'Del = ' + Del.show

DelPerms = []
perm = Del.dup
(Del.size).times do |n|
  perm = perm.cyclic_permutation
  DelPerms << perm << perm.inverse
end
DelPerms.freeze
p DelPerms.map(&:show)
nil

"Del = (((abAB)(cdCD))(efEF))(s)"
["bABcdCDefEFsa", "ASfeFEdcDCbaB", "ABcdCDefEFsab", "BASfeFEdcDCba", "BcdCDefEFsabA", "aBASfeFEdcDCb", "cdCDefEFsabAB", "baBASfeFEdcDC", "dCDefEFsabABc", "CbaBASfeFEdcD", "CDefEFsabABcd", "DCbaBASfeFEdc", "DefEFsabABcdC", "cDCbaBASfeFEd", "efEFsabABcdCD", "dcDCbaBASfeFE", "fEFsabABcdCDe", "EdcDCbaBASfeF", "EFsabABcdCDef", "FEdcDCbaBASfe", "FsabABcdCDefE", "eFEdcDCbaBASf", "sabABcdCDefEF", "feFEdcDCbaBAS", "abABcdCDefEFs", "SfeFEdcDCbaBA"]


In [5]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [355]:
#myw = Word.new(a[1], b[1], a[1].inverse)
#myw = comms[1]
myw = Word.new(a[1])*(comms[2]^2)
p myw.show

"(a)((cdCD)(cdCD))"


"(a)((cdCD)(cdCD))"

In [356]:
box = {}
Rseq.each{|x| box.store(x.char,[])}

facs = myw.flatten.factors

facs.each_with_index do |c, i|
  k = i+1
  box[c.char].insert(-1, 2*k-1)
  box[c.inverse.char].insert(0,2*k)
end
p box, box.values, box.values.map(&:to_s).join
nil

{"a"=>[1], "B"=>[], "A"=>[2], "b"=>[], "c"=>[16, 8, 3, 11], "D"=>[14, 6, 9, 17], "C"=>[12, 4, 7, 15], "d"=>[18, 10, 5, 13], "e"=>[], "F"=>[], "E"=>[], "f"=>[]}
[[1], [], [2], [], [16, 8, 3, 11], [14, 6, 9, 17], [12, 4, 7, 15], [18, 10, 5, 13], [], [], [], []]
"[1][][2][][16, 8, 3, 11][14, 6, 9, 17][12, 4, 7, 15][18, 10, 5, 13][][][][]"


In [357]:
p box.values.map(&:to_s).join

coprods = []
box.values.flatten.each_cons(2) do |pair|
  if (pair[0].even? and pair[1].odd?) then
    elm1, elm2 = {}, {}
    order = (pair[0] < pair[1])
    num = (order) ? [pair[0], pair[1]-1] : [pair[1]-1, pair[0]]
    box.each_pair do |key, arr|
      # separate the box
      a1 = arr.select{|x| x <= num[0] or num[1] < x}
      a2 = arr.select{|x| num[0] < x and x <= num[1]}
      # shift numbers
      a1.map!{|x| (num[1]<x) ? x-(num[1]-num[0]) : x}
      a2.map!{|x| x-num[0]}
      # store values into the boxes
      elm1.store(key, a1)
      elm2.store(key, a2)
    end
    # contract elm1 & elm2
    # ---
    # 
    coprods << {pair: pair, sign: (order) ? 1 : -1, elm1: elm1, elm2: elm2}
  end
end
  
printf("----------------------\n")
coprods.each do|h|
  str1 = h[:elm1].values.map(&:to_s).join
  str2 = h[:elm2].values.map(&:to_s).join
  p "#{h[:pair]} : #{h[:sign]}, " + str1 + " (x) " + str2
end
nil

"[1][][2][][16, 8, 3, 11][14, 6, 9, 17][12, 4, 7, 15][18, 10, 5, 13][][][][]"
----------------------
"[8, 3] : -1, [1][][2][][10, 5][8, 3, 11][6, 9][12, 4, 7][][][][] (x) [][][][][6, 1][4][2, 5][3][][][][]"
"[6, 9] : 1, [1][][2][][14, 3, 9][12, 6, 7, 15][10, 4, 13][16, 8, 5, 11][][][][] (x) [][][][][2][][1][][][][][]"
"[4, 7] : 1, [1][][2][][14, 6, 3, 9][12, 7, 15][10, 4, 5, 13][16, 8, 11][][][][] (x) [][][][][][2][][1][][][][]"
"[10, 5] : -1, [1][][2][][10, 3, 5][8, 11][6, 4, 9][12, 7][][][][] (x) [][][][][4][2, 5][3][6, 1][][][][]"


In [358]:
p box.values
bvf = box.values.flatten
max = bvf.max

evens=(1..max).to_a.combination(2).select{|pair| pair[0].even? and pair[1].even?}
evens.each do |pair|
  cons = [[pair[0], pair[0]+1], [pair[1], (pair[1]<max) ? pair[1]+1 : 1]]
  order = bvf.select{|n| cons.flatten.include?(n)}
  p "#{cons} --> #{order} : #{order.map{|e| cons.flatten.index(e)}}"
end
nil

[[1], [], [2], [], [16, 8, 3, 11], [14, 6, 9, 17], [12, 4, 7, 15], [18, 10, 5, 13], [], [], [], []]
"[[2, 3], [4, 5]] --> [2, 3, 4, 5] : [0, 1, 2, 3]"
"[[2, 3], [6, 7]] --> [2, 3, 6, 7] : [0, 1, 2, 3]"
"[[2, 3], [8, 9]] --> [2, 8, 3, 9] : [0, 2, 1, 3]"
"[[2, 3], [10, 11]] --> [2, 3, 11, 10] : [0, 1, 3, 2]"
"[[2, 3], [12, 13]] --> [2, 3, 12, 13] : [0, 1, 2, 3]"
"[[2, 3], [14, 15]] --> [2, 3, 14, 15] : [0, 1, 2, 3]"
"[[2, 3], [16, 17]] --> [2, 16, 3, 17] : [0, 2, 1, 3]"
"[[2, 3], [18, 1]] --> [1, 2, 3, 18] : [3, 0, 1, 2]"
"[[4, 5], [6, 7]] --> [6, 4, 7, 5] : [2, 0, 3, 1]"
"[[4, 5], [8, 9]] --> [8, 9, 4, 5] : [2, 3, 0, 1]"
"[[4, 5], [10, 11]] --> [11, 4, 10, 5] : [3, 0, 2, 1]"
"[[4, 5], [12, 13]] --> [12, 4, 5, 13] : [2, 0, 1, 3]"
"[[4, 5], [14, 15]] --> [14, 4, 15, 5] : [2, 0, 3, 1]"
"[[4, 5], [16, 17]] --> [16, 17, 4, 5] : [2, 3, 0, 1]"
"[[4, 5], [18, 1]] --> [1, 4, 18, 5] : [3, 0, 2, 1]"
"[[6, 7], [8, 9]] --> [8, 6, 9, 7] : [2, 0, 3, 1]"
"[[6, 7], [10, 11]] --> [11, 6, 7, 10] : [3, 0, 